In [1]:
# Importing google drive to colab
from google.colab import drive
drive.mount('/content/gdrive')
# to change the current working directory for the notebook environment
%cd '/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/SciFair/NLP


In [2]:
!pip install numpy
!pip install nltk
!pip install tensorflow

!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing
!pip install Pillow==9.5.0

  Cloning https://github.com/MihaMarkic/tflearn.git (to revision fix/is_sequence_missing) to /tmp/pip-req-build-dg_24xk4
  Running command git clone --filter=blob:none --quiet https://github.com/MihaMarkic/tflearn.git /tmp/pip-req-build-dg_24xk4
  Running command git checkout -b fix/is_sequence_missing --track origin/fix/is_sequence_missing
  Switched to a new branch 'fix/is_sequence_missing'
  Branch 'fix/is_sequence_missing' set up to track remote branch 'fix/is_sequence_missing' from 'origin'.
  Resolved https://github.com/MihaMarkic/tflearn.git to commit 6472b8588e758ff4a33a2764d4ee638bbd0e42f0
  Preparing metadata (setup.py) ... done


In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import numpy
# import tflearn #only works in local
import tensorflow
import random
import json
with open('intents_ZH-TW.json') as file:
    data=json.load(file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
'''
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
'''
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append (intent["tag"])


In [5]:
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [6]:
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

In [7]:
training=numpy.array(training)
output=numpy.array(output)

In [ ]:
# !pip uninstall tflearn
# !pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing

In [ ]:
# !pip install Pillow==9.5.0

In [8]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 17999  | total loss: 0.00003 | time: 0.074s
| Adam | epoch: 1500 | loss: 0.00003 - acc: 1.0000 -- iter: 88/93
Training Step: 18000  | total loss: 0.00003 | time: 0.078s
| Adam | epoch: 1500 | loss: 0.00003 - acc: 1.0000 -- iter: 93/93
--


In [10]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [11]:
!apt-get update && apt-get install -y libasound2-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,091 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Pack

In [12]:
!pip install SpeechRecognition
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 58.4 MB/s eta 0:00:00


In [14]:
!pip install ffmpeg-python

In [40]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")

  # Check if data contains a comma before splitting
  if ',' in data:
    binary = b64decode(data.split(',')[1])
  else:
    # Handle the case where data doesn't contain a comma
    # For example, you might want to raise an exception or return an error value
    # Here, we're assuming the data is already base64 encoded and doesn't need splitting
    binary = b64decode(data)

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [19]:
audio, sr = get_audio()


In [20]:
!pip install whisper
!pip install sounddevice wavio
!pip install git+https://github.com/openai/whisper.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=521cd882bbadd9a6dec1ebf066b33f804af7158a436150aa9f1214b2c79de33d
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
Successfully built whisper
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5r_bw5e6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5r_bw5e6
  Resolved https://github.com/openai/whisper.git to commit 5979f03701209bb035a0a466f14131aeb1116cbb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.1 MB/s eta 0:00:00


In [38]:
!pip install gtts
from gtts import gTTS
import os
import time

In [48]:
import random
import time
import numpy as np
import whisper
import sounddevice as sd
import wavio
import os


# Load Whisper model (you can choose 'base', 'small', 'medium', 'large')
whisper_model = whisper.load_model("base")

# Function to record audio from the microphone
def record_audio(duration=5):
    print(f"Recording for {duration} seconds...")

    # Call get_audio to record audio
    audio, sr = get_audio()

    # Save the recorded audio as a WAV file
    filename = "recorded_audio.wav"
    wavio.write(filename, audio, sr, sampwidth=3)  # Save as WAV file

    return filename


# Function to transcribe the recorded audio using Whisper
def transcribe_audio(filename):
    result = whisper_model.transcribe(filename, language='zh')  # Specify Chinese language
    return result['text']

def speak(text):
    tts = gTTS(text=text, lang='zh', slow=False)  # Adjust 'lang' as needed
    filename = "temp_audio.mp3"
    tts.save(filename)
    os.system(f"start {filename}")  # For Windows; use 'xdg-open' for Linux, 'afplay' for macOS

# Main chat function
def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    speak("開始與您的機器人對話，輸入結束以停止")
    time.sleep(2)

    while True:
        try:
            choice = input("輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：")
            if choice == '1':  # Text input
                inp = input("請輸入您的問題：")
            elif choice == '2':  # Speech input
                filename = record_audio(duration=5)
                inp = transcribe_audio(filename)  # Transcribe the recorded audio
                print("You (語音):", inp)
            else:
                print("無效的選擇，請輸入 '1' 或 '2'。")
                continue  # Skip to the next iteration

            time.sleep(2)
            if inp == "結束":
                speak("再見！")
                print("再見！")
                break

            # Process the input for the chatbot
            results = model.predict([bag_of_words(inp, words)])
            results_index = np.argmax(results)
            tag = labels[results_index]

            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            rd = random.choice(responses)
            speak(rd)  # Speak the response
            print(rd)
            time.sleep(2)

        except Exception as e:
            speak("抱歉，發生了錯誤")
            print("發生錯誤:", e)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [49]:
chat()

開始與您的機器人對話（輸入 結束 以停止）：
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：2
Recording for 5 seconds...


You (語音): 
親密關係中的問題可能對心理健康有重大影響。你是否願意分享更多關於這方面的情況？
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：2
Recording for 5 seconds...


You (語音): 你好,請問你可以怎麼幫我
你好，有什麼需要我協助的嗎？
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：2
Recording for 5 seconds...


You (語音): 祟祝祚
家人和朋友之間的關係可能是引起情緒困擾的一個因素。我願意聽取更多你的故事。
輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：2
Recording for 5 seconds...


You (語音): 結束
再見！


In [47]:
# def chat():
#     print("開始與您的機器人對話（輸入 結束 以停止）：")
#     # speak("開始與您的機器人對話（輸入 結束 以停止）：")
#     choice = input("輸入 '1' 以進行文字輸入，或 '2' 以進行語音輸入：")
#     time.sleep(5)
#     while True:
#         try:
#             if choice == '1':  # Text input
#                 inp = input("請輸入您的問題：")
#                 if inp=="結束":
#                   break
#             elif choice == '2':  # Speech input
#               inp=listenTo()
#               print("You:",inp)
#               time.sleep(2)
#               if inp=="結束":
#                   break
#               print("You (語音):", inp)
#             else:
#               print("無效的選擇，請輸入 '1' 或 '2'。")
#               continue  # Skip to the next iteration


#             results=model.predict([bag_of_words(inp,words)])
#             results_index=numpy.argmax(results)
#             tag=labels[results_index]

#             for tg in data["intents"]:
#                 if tg["tag"]==tag:
#                     responses=tg["responses"]
#             rd=random.choice(responses)
#             #speak(rd)
#             print(rd)
#             time.sleep(5)
#         except sr.UnknownValueError:
#             print("我無法理解音訊")
#         except sr.RequestError as e:
#             print("無法從 Google 語音識別服務請求結果; {0}".format(e))
# chat()

In [46]:
# def chat():
#     print("開始與您的機器人對話（輸入 結束 以停止）：")
#     speak("開始與您的機器人對話（輸入 結束 以停止）：")
#     time.sleep(5)
#     while True:
#         try:
#             inp=listenTo()
#             print("You:",inp)
#             time.sleep(2)
#             if inp=="結束":
#                 break
#             results=model.predict([bag_of_words(inp,words)])
#             results_index=numpy.argmax(results)
#             tag=labels[results_index]

#             for tg in data["intents"]:
#                 if tg["tag"]==tag:
#                     responses=tg["responses"]
#             rd=random.choice(responses)
#             speak(rd)
#             print(rd)
#             time.sleep(5)
#         except sr.UnknownValueError:
#             print("我無法理解音訊")
#         except sr.RequestError as e:
#             print("無法從 Google 語音識別服務請求結果; {0}".format(e))
# chat()